# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv = "../python-api-challenge/output/cities.csv"

city_weather = pd.read_csv(csv)

city_weather.head()

,City,City.1,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nha trang,12.2500,109.1833,84.20,81,100,3.62,VN,2022-08-12
1,1,kapaa,22.0752,-159.3190,73.72,91,40,11.50,US,2022-08-12
2,2,sitka,57.0531,-135.3300,60.69,88,40,0.00,US,2022-08-12
3,3,punta arenas,-53.1500,-70.9167,35.71,87,100,6.91,CL,2022-08-12
4,4,ponta do sol,32.6667,-17.1000,79.00,61,19,5.37,PT,2022-08-12


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_weather[['Lat', 'Lon']]
weights = city_weather['Humidity']

layout = {
    'width':'100%',
    'height':'400px',
    'border':'1px sold black',
    'padding': '1px'
}
fig = gmaps.figure(layout=layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=100, point_radius= 1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(border='1px sold black', height='400px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_weather = city_weather.loc[
    (city_weather['Max Temp']> 70) &
    (city_weather['Max Temp']<90) &
    (city_weather['Humidity']<70) &
    (city_weather['Cloudiness']<50) &
    (city_weather['Wind Speed']<20) &
    (city_weather['Country'] != 'RU') # Putin sucks
]
ideal_weather.reset_index()
ideal_weather.head()

,City,City.1,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,ponta do sol,32.6667,-17.1000,79.00,61,19,5.37,PT,2022-08-12
16,16,palanga,55.9175,21.0686,77.02,47,0,4.61,LT,2022-08-12
26,26,saint-avold,49.1027,6.7006,88.79,25,0,12.33,FR,2022-08-12
40,40,camacha,33.0833,-16.3333,74.62,61,40,10.36,PT,2022-08-12
48,48,sinnamary,5.3833,-52.9500,85.12,63,0,6.33,GF,2022-08-12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = ideal_weather[['City', 'Lat', 'Lon', 'Country']].copy()
hotel_df['Hotel Name'] = np.nan
hotel_df = hotel_df.rename(
    columns={
        'Lon':'Lng'
    }
)
hotel_df.head()

,City,Lat,Lng,Country,Hotel Name
4,4,32.6667,-17.1000,PT,NaN
16,16,55.9175,21.0686,LT,NaN
26,26,49.1027,6.7006,FR,NaN
40,40,33.0833,-16.3333,PT,NaN
48,48,5.3833,-52.9500,GF,NaN


In [6]:
ideal_weather.head()

,City,City.1,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,ponta do sol,32.6667,-17.1000,79.00,61,19,5.37,PT,2022-08-12
16,16,palanga,55.9175,21.0686,77.02,47,0,4.61,LT,2022-08-12
26,26,saint-avold,49.1027,6.7006,88.79,25,0,12.33,FR,2022-08-12
40,40,camacha,33.0833,-16.3333,74.62,61,40,10.36,PT,2022-08-12
48,48,sinnamary,5.3833,-52.9500,85.12,63,0,6.33,GF,2022-08-12


In [7]:
# I need a place to sleep after all of my fun
target_search = 'hotel'
target_radius = 5000
target_type = 'lodging'

params = {
    'keyword': target_search,
    'radius': target_radius,
    'type': target_type,
    'key': g_key
}
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df[0:10].iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat}, {lng}"

    hotel_data = requests.get(url, params).json()

# Save the first hotel name for each city
    try:
        hotel_name = hotel_data['results'][0]['name']
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
        print(f"City '{row['City']}' found: The nearest hotel is {hotel_name} ")
        print("----------------------")
    except:
        print(f"Couldn't find any information for city '{row['City']}' . . . skipping.") 
        print("----------------------")
        print("Finished")

City '4' found: The nearest hotel is Enotel Sunset Bay 
----------------------
City '16' found: The nearest hotel is Palanga Spa Luxury 
----------------------
City '26' found: The nearest hotel is Novotel Saint-Avold 
----------------------
City '40' found: The nearest hotel is Vila Baleira Resort 
----------------------
City '48' found: The nearest hotel is Hotel Du Fleuve 
----------------------
Couldn't find any information for city '54' . . . skipping.
----------------------
Finished
City '64' found: The nearest hotel is The Advenire, Autograph Collection 
----------------------
City '72' found: The nearest hotel is Kansa Hotel 
----------------------
City '111' found: The nearest hotel is Tanna Ikamir Guest House 
----------------------
City '130' found: The nearest hotel is Quality Suites Vila Velha 
----------------------


In [8]:
hotel_df.dropna(how="any", inplace=True)
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Lat,Lng,Country,Hotel Name
0,4,32.6667,-17.1000,PT,Enotel Sunset Bay
1,16,55.9175,21.0686,LT,Palanga Spa Luxury
2,26,49.1027,6.7006,FR,Novotel Saint-Avold
3,40,33.0833,-16.3333,PT,Vila Baleira Resort
4,48,5.3833,-52.9500,GF,Hotel Du Fleuve
5,64,37.1041,-113.5841,US,"The Advenire, Autograph Collection"
6,72,-17.8786,36.8883,MZ,Kansa Hotel
7,111,-19.5500,169.2667,VU,Tanna Ikamir Guest House
8,130,-20.3297,-40.2925,BR,Quality Suites Vila Velha


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px sold black', height='400px', padding='1px', width='100%'))